# Word Representation in Biomedical Domain

Before you start, please make sure you have read this notebook. You are encouraged to follow the recommendations but you are also free to develop your own solution from scratch. 

## Marking Scheme

- Biomedical imaging project: 40%
    - 20%: accuracy of the final model on the test set
    - 20%: rationale of model design and final report
- Natural language processing project: 40%
    - 30%: completeness of the project
    - 10%: final report
- Presentation skills and team work: 20%


This project forms 40\% of the total score for summer/winter school. The marking scheme of each part of this project is provided below with a cap of 100\%.

You are allowed to use open source libraries as long as the libraries are properly cited in the code and final report. The usage of third-party code without proper reference will be treated as plagiarism, which will not be tolerated.

You are encouraged to develop the algorithms by yourselves (without using third-party code as much as possible). We will factor such effort into the marking process.

## Setup and Prerequisites 

Recommended environment

- Python 3.7 or newer
- Free disk space: 100GB

Download the data

```sh
# navigate to the data folder
cd data

# download the data file
# which is also available at https://www.semanticscholar.org/cord19/download
wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2021-07-26/document_parses.tar.gz

# decompress the file which may take several minutes
tar -xf document_parses.tar.gz

# which creates a folder named document_parses
```

## Part 1 (20%): Parse the Data

The JSON files are located in two sub-folders in `document_parses`. You will need to scan all JSON files and extract text (i.e. `string`) from relevant fields (e.g. body text, abstract, titles).

You are encouraged to extract full article text from body text if possible. If the hardware resource is limited, you can extract from abstract or titles as alternatives. 

Note: The number of JSON files is around 425k so it may take more than 10 minutes to parse all documents.

For more information about the dataset: https://www.semanticscholar.org/cord19/download

Recommended output:

- A list of text (`string`) extracted from JSON files.

In [3]:
###################
# TODO: add your solution
import os
import json
from tqdm import tqdm

def parse_data() -> list[str]:
    """ Scans all JSON files in folder `data` and extracts full text from them.

    Returns:
       list[str]: A list containing full article text of all papers.
    """    

    res = []

    doc_dir = os.path.join("data", "document_parses")
    pdf_dir = os.path.join(doc_dir, "pdf_json")
    pmc_dir = os.path.join(doc_dir, "pmc_json")
    file_paths = [os.path.join(pdf_dir, filename) for filename in os.listdir(pdf_dir)]
    file_paths.extend([os.path.join(pmc_dir, filename) for filename in os.listdir(pmc_dir)])
    
    with tqdm(file_paths, desc="Parsing papers", unit="paper") as pbar:
        for file_path in pbar:
            with open(file_path, encoding='utf-8') as f:
                paper = json.load(f)
                body_text = paper['body_text']
                res.append('\n'.join(para['text'] for para in body_text))

    return res

full_texts = parse_data()
###################

Parsing papers: 100%|██████████| 425257/425257 [13:20<00:00, 531.52paper/s] 


In [3]:
print(len(full_texts))
print(full_texts[0])

425257
Digital technologies have provided support in diverse policy, business, and societal application areas in the COVID-19 outbreak, such as pandemic management (Radanliev et al., 2020b) , corporate communications (Camilleri, 2020) , analysis of research data (Radanliev et al., 2020a) , and education (Crawford et al., 2020) . COVID-19 started as a global infectious disease in the spring of 2020, but the necessary measures to control the virus went beyond treatment and were also directed against its spread. Thus, for months, all interpersonal relationships were characterized by social distancing, and the pandemic raised not only medical but also social, economic and technological issues, among others. Higher education was one domain that the pandemic affected radically (Nuere and de Miguel, 2020; Watermeyer et al., 2020) . During the worldwide lockdown, higher educational institutions had to immediately switch their activities from the classroom and the campus to a virtual space, whi

## Part 2 (30%): Tokenization

Traverse the extracted text and segment the text into words (or tokens).

The following tracks can be developed in independentely. You are encouraged to divide the workload to each team member.

Recommended output:

- Tokenizer(s) that is able to tokenize any input text.

Note: Because of the computation complexity of tokenizers, it may take hours/days to process all documents. Which tokenizer is more efficient? Any idea to speedup?

### Track 2.1 (10%): Use split()

Use the standard `split()` by Python.

### Track 2.2 (10%): Use NLTK or SciSpaCy

NLTK tokenizer: https://www.nltk.org/api/nltk.tokenize.html

SciSpaCy: https://github.com/allenai/scispacy

Note: You may need to install NLTK and SpaCy so please refer to their websites for installation instructions.

### Track 2.3 (10%): Use Byte-Pair Encoding (BPE)

Byte-Pair Encoding (BPE): https://huggingface.co/transformers/tokenizer_summary.html

Note: You may need to install Huggingface's transformers so please refer to its website for installation instructions.

### Track 2.4 (Bonus +5%): Build new Byte-Pair Encoding (BPE)

This track may be dependent on track 2.3.

The above pre-built tokenization methods may not be suitable for biomedical domain as the words/tokens (e.g. diseases, sympotoms, chemicals, medications, phenotypes, genotypes etc.) can be very different from the words/tokens commonly used in daily life. Can you build and train a new BPE model for biomedical domain in particular?

### Open Question (Optional):

- What are the pros and cons of the above tokenizers?

In [14]:
###################
# TODO: add your solution

# Track 2.1

import re
from gensim import corpora

def tokenizer_1():
    res = []
    for text in tqdm(full_texts, desc="tokenize", unit="text"):
        res.append([word for word in re.split(r'\W+', text.lower()) if word != ''])
    return res

texts = tokenizer_1()
dictionary = corpora.Dictionary(texts)
dictionary.save('token.dict')  # store the dictionary, for future reference
print(dictionary)

tokenize:   0%|          | 0/425257 [00:00<?, ?text/s]

Dictionary<2031501 unique tokens: ['0', '001', '01', '1', '12']...>


In [ ]:
print(dictionary.doc2idx(re.split(r'\W+', full_texts[0].lower())))

In [ ]:
"""
TODO: token2id id2token
"""

class TokenizerSplit(object):

    def __init__(self):
        self.dictionary = None

    def tokenize(self, in_texts):
        res = []
        for text in tqdm(in_texts, desc="tokenize", unit="text"):
            res.append([word for word in re.split(r'\W+', text.lower()) if word != ''])
        self.dictionary = corpora.Dictionary(res)

    def encode(self, text):
        return self.dictionary.doc2idx([word for word in re.split(r'\W+', text.lower()) if word != ''])

    def decode(self, cor):
        text = ''
        for token_id in cor:
            text += dictionary.id2token(cor)
        return text

    def getTokenId(self, word):
        return dictionary.token2id(word)


In [11]:
# Track 2.2

""""
install nltk, run for one time
"""

import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/zhangyuechen/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/zhangyuechen/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/zhangyuechen/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/zhangyuechen/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/zhangyuechen/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_

True

In [ ]:
from nltk.tokenize import word_tokenize

def tokenizer_2():
    res = []
    for text in tqdm(full_texts, desc="tokenize", unit="text"):
        res.append([word for word in word_tokenize(text.lower()) if word != ''])
    return res

texts_2 = tokenizer_2()
dictionary_2 = corpora.Dictionary(texts_2)
dictionary_2.save('token_2.dict')  # store the dictionary, for future reference
print(dictionary)


In [ ]:
class TokenizerNLTK(object):

    def __init__(self):
        self.dictionary = None

    def tokenize(self, in_texts):
        res = []
        for text in tqdm(in_texts, desc="tokenize", unit="text"):
            res.append([word for word in word_tokenize(text.lower()) if word != ''])
        self.dictionary = corpora.Dictionary(res)

    def encode(self, text):
        return self.dictionary.doc2idx([word for word in re.split(r'\W+', text.lower()) if word != ''])

    def decode(self, cor):
        text = ''
        for token_id in cor:
            text += dictionary.id2token(cor)
        return text

    def getTokenId(self, word):
        return dictionary.token2id(word)


In [ ]:
###################
# Track 2.3
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"], vocab_size=20000)
tokenizer.pre_tokenizer = Whitespace()
tokenizer.train_from_iterator(full_texts, trainer)
###################

In [ ]:
tokenizer.save("BPETokenizer.json")

In [ ]:
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_file("BPETokenizer.json")

# print(tokenizer.get_vocab())
print(tokenizer.get_vocab_size())
print(tokenizer.id_to_token(19824))
print(tokenizer.token_to_id("tees"))

# output = tokenizer.encode(full_texts[0])
# print(output.tokens)
# print(tokenizer.decode(output.ids))

print(tokenizer.encode("diseases").tokens)
print(tokenizer.encode("symptoms").tokens)
print(tokenizer.encode("chemicals").tokens)
print(tokenizer.encode("medications").tokens)
print(tokenizer.encode("phenotypes").tokens)
print(tokenizer.encode("genotypes").tokens)

In [2]:
###################
# Track 2.3 MY BPE
import re
from typing import Dict, List, Tuple

TokenizationState = Dict[str, int]
""" State of tokenization of text.  
E.g., `{'i</w>': 2, 'a m </w>': 1, 'ha pp y </w>': 1, 'ha v e </w>': 1, 'a pp l e s </w>': 1 }`
"""
Tokens = List[Tuple[str, int]]
""" Tokens along with their frequencies.
E.g, `[('i</w>', 2), ('ha', 2), ('pp', 2), ('a', 2), ('m', 1), ('</w>', 5), ('y', 1), ('v', 1), ('e', 2), ('l', 1), ('s', 1)]`
"""
Bigram = Tuple[str]
""" One bigram.
E.g., ('ha', 'pp').
"""

class MyBPETokenizer:
    def __init__(self, vocab_size=30000):
        self.vocab_size = vocab_size
        self.token2id = dict()
        self.id2token = dict()

    def token_to_id(self, token: str) -> int:
        return self.token2id[token]

    def id_to_token(self, id: int) -> str:
        return self.id2token[id]

    def encode(self):
        pass

    def decode(self):
        pass

    def pre_tokenize(self, corpus):
        _splitor_pattern = re.compile(R"[^a-zA-Z']+|(?=')")

        corpus = [list(
            filter(lambda tkn: len(tkn)>0, _splitor_pattern.split(stc.lower()))) for stc in corpus
        ]
        
        return corpus

    def build_bpe_vocab(self, corpus: List[str]) -> TokenizationState:
        """ Pre-tokenizes text and build the initial tokenization state.

        Args:
            corpus (List[str]): Text to tokenize.

        Returns:
            TokenizationState: Initial tokenization state (characters split by whitespace and words appended by `</w>`).
        """        

        tokenized_corpus = self.pre_tokenize(corpus)
        bpe_vocab = dict()
        for stc in tokenized_corpus:
            for word in stc:
                key = ' '.join(list(word)) + ' </w>'
                bpe_vocab[key] = bpe_vocab.get(key, 0) + 1
        return bpe_vocab

    def get_bigram_freq(self, bpe_vocab: TokenizationState) -> dict[Bigram, int]:
        """ Counts frequencies of bigrams in current tokenization state.

        Args:
            bpe_vocab (TokenizationState): Current tokenization state.

        Returns:
            dict[Bigram, int]: A dictionary from bigrams to their frequencies.
        """        

        bigram_freq = dict()
        for word, count in bpe_vocab.items():
            word = word.split(' ')
            bigrams = [(word[i], word[i+1]) for i in range(len(word)-1)]
            for bigram in bigrams:
                bigram_freq[bigram] = bigram_freq.get(bigram, 0) + count
        return bigram_freq

    def refresh_bpe_vocab_by_merging_bigram(self, bigram: Bigram, old_bpe_vocab: TokenizationState) -> TokenizationState:
        """ Updates tokenization state by merging given bigram (removing whitespace).

        Args:
            bigram (Tuple): The bigram to merge.
            old_bpe_vocab (TokenizationState): Old tokenization state.

        Returns:
            TokenizationState: New tokenization state after merging.
        """        

        new_bpe_vocab = dict()
        bigram_pat = re.compile(f"((?<= )|^){bigram[0]} {bigram[1]}((?= )|$)")
        new_token = ''.join(bigram)
        for old_word, count in old_bpe_vocab.items():
            new_word = re.sub(bigram_pat, new_token, old_word)
            new_bpe_vocab[new_word] = new_bpe_vocab.get(new_word, 0) + count
        return new_bpe_vocab
    
    def get_bpe_tokens(self, bpe_vocab: TokenizationState) -> Tokens:
        """ Returns the list of tokens along with their frequencies according to 
        the dictionary from state of tokenization to frequencies.

        Args:
            bpe_vocab (TokenizationState): The dictionary from state of tokenization to frequencies.

        Returns:
            Tokens: List of tokens along with their frequencies.
        """        

        bpe_tokens_dict = dict()
        for word, count in bpe_vocab.items():
            tokens = word.split(' ')
            for token in tokens:
                bpe_tokens_dict[token] = bpe_tokens_dict.get(token, 0) + count

        bpe_tokens = [(token, freq) for token, freq in bpe_tokens_dict.items()]
        bpe_tokens.sort(  # sort by length, note </w> contributes only 1 to length
            key = lambda itm: len(itm[0]) if '</w>' not in itm[0] else len(itm[0])-3,
            reverse = True)

        return bpe_tokens

    def tokenize_subword(self, subword: str) -> List[str]:
        """ Recursively splits `subword` into tokens.

        Args:
            sub_word (str): The subword to tokenize.

        Returns:
            List[str]: Tokens of given subword.
        """        
        if subword == '':
            return []

        for token in self.token2id:
            if token in subword:
                res = token
                left = self.tokenize_subword(subword[ : subword.index(token)])
                right = self.tokenize_subword(subword[subword.index(token) + len(token) : ])
                return left + [res] + right
        
        return ['<UNK>']

    def tokenize_text(self, sentence: List[str]) -> List[str]:
        """ Tokenizes text.

        Args:
            sentence (List[str]): The text to be tokenize.

        Returns:
            List[str]: Tokens of given text.
        """        

        res = []
        for word in sentence:
            res.extend(self.tokenize_subword(word + "</w>"))
        return res
        
    def train(self, corpus):
        """ Trains on `corpus`, that is, constructs `token2id` and `id2token`.

        Args:
            corpus (List[str]): The training corpus.
        """        

        training_bpe_vocab = self.build_bpe_vocab(corpus)
        for i in range(self.vocab_size):
            bigram_freq = self.get_bigram_freq(training_bpe_vocab)

            if bigram_freq:
                updated_bigram = max(bigram_freq, key=bigram_freq.get)
            else:  # no bigrams left
                print(f"No subwords left after {i} merges!")
                break
            
            training_bpe_vocab = self.refresh_bpe_vocab_by_merging_bigram(
                updated_bigram, training_bpe_vocab)

        training_bpe_tokens = self.get_bpe_tokens(training_bpe_vocab)
        
        for token in training_bpe_tokens:
            token = token[0]
            idx = len(self.token2id)
            self.token2id[token], self.id2token[idx] = idx, token

###################

In [3]:
bpe_tokenizer = MyBPETokenizer(vocab_size=10)
bpe_tokenizer.train(["I am happy.", "I have 10 apples!"])
print(bpe_tokenizer.token2id)
print(bpe_tokenizer.tokenize_text(["happy", "apples"]))

{'happy</w>': 0, 'have': 1, 'am</w>': 2, 'i</w>': 3, 'pp': 4, '</w>': 5, 'a': 6, 'l': 7, 'e': 8, 's': 9}
['happy</w>', 'a', 'pp', 'l', 'e', 's', '</w>']



## Part 3 (30%): Build Word Representations

Build word representations for each extracted word. If the hardware resource is limited, you may limit the vocabulary size up to 10k words/tokens (or even smaller) and the dimension of representations up to 256.

The following tracks can be developed independently. You are encouraged to divide the workload to each team member.

### Track 3.1 (15%): Use N-gram Language Modeling

N-gram Language Modeling is to predict a target word by using `n` words from previous context. Specifically,

$P(w_i | w_{i-1}, w_{i-2}, ..., w_{i-n+1})$

For example, given a sentence, `"the main symptoms of COVID-19 are fever and cough"`, if $n=7$, we use previous context `["the", "main", "symptoms", "of", "COVID-19", "are"]` to predict the next word `"fever"`.

More to read: https://web.stanford.edu/~jurafsky/slp3/3.pdf

Recommended outputs:

- A fixed vector for each word/token.

### Track 3.2 (15%): Use Skip-gram with Negative Sampling

In skip-gram, we use a central word to predict its context. Specifically,

$P(w_{c-m}, ... w_{c-1}, w_{c+1}, ..., w_{c+m} | w_c)$

As the learning objective of skip-gram is computational inefficient (summation of entire vocabulary $|V|$), negative sampling is commonly applied to accelerate the training.

In negative sampling, we randomly select one word from the context as a positive sample, and randomly select $K$ words from the vocabulary as negative samples. As a result, the learning objective is updated to

$L = -\log\sigma(u^T_{t} v_c) - \sum_{k=1}^K\log\sigma(-u^T_k v_c)$, where $u_t$ is the vector embedding of positive sample from context, $u_k$ are the vector embeddings of negative samples, $v_c$ is the vector embedding of the central word, $\sigma$ refers to the sigmoid function.

More to read http://web.stanford.edu/class/cs224n/readings/cs224n-2019-notes01-wordvecs1.pdf section 4.3 and 4.4

Recommended outputs:

- A fixed vector for each word/token.

### Track 3.3 (Bonus +5%): Use Contextualised Word Representation by Masked Language Model (MLM)

BERT introduces a new language model for pre-training named Masked Language Model (MLM). The advantage of MLM is that the word representations by MLM will be contextualised.

For example, "stick" may have different meanings in different context. By N-gram language modeling and word2vec (skip-gram, CBOW), the word representation of "stick" is fixed regardless of its context. However, MLM will learn the representation of "stick" dynamatically based on context. In other words, "stick" will have different representations in different context by MLM.

More to read: http://jalammar.github.io/illustrated-bert/ and https://arxiv.org/pdf/1810.04805.pdf

Recommended outputs:

- An algorithm that is able to generate contextualised representation in real time.

In [3]:
###################
# TODO: add your solution

###################

## Part 4 (20%): Explore the Word Representations

The following tracks can be finished independently. You are encouraged to divide workload to each team member.

### Track 4.1 (5%): Visualise the word representations by t-SNE

t-SNE is an algorithm to reduce dimentionality and commonly used to visualise high-dimension vectors. Use t-SNE to visualise the word representations. You may visualise up to 1000 words as t-SNE is highly computationally complex.

More about t-SNE: https://lvdmaaten.github.io/tsne/

Recommended output:

- A diagram by t-SNE based on representations of up to 1000 words.

### Track 4.2 (5%): Visualise the Word Representations of Biomedical Entities by t-SNE

Instead of visualising the word representations of the entire vocabulary (or 1000 words that are selected at random), visualise the word representations of words which are biomedical entities. For example, fever, cough, diabetes etc. Based on the category of those biomedical entities, can you assign different colours to the entities and see if the entities from the same category can be clustered by t-SNE? For example, sinusitis and cough are both respirtory diseases so they should be assigned with the same colour and ideally their representations should be close to each other by t-SNE. Another example, Alzheimer and headache are neuralogical diseases which should be assigned by another colour.

Examples of biomedial ontology: https://www.ebi.ac.uk/ols/ontologies/hp and https://en.wikipedia.org/wiki/International_Classification_of_Diseases

Recommended output:

- A diagram with colours by t-SNE based on representations of biomedical entities.

### Track 4.3 (5%): Co-occurrence

- What are the biomedical entities which frequently co-occur with COVID-19 (or coronavirus)?

Recommended outputs:

- A sorted list of biomedical entities and description on how the entities are selected and sorted.

### Track 4.4 (5%): Semantic Similarity

- What are the biomedical entities which have closest semantic similarity COVID-19 (or coronavirus) based on word representations?

Recommended outputs:

- A sorted list of biomedical entities and description on how the entities are selected and sorted.

### Open Question (Optional): What else can you discover?


In [4]:
###################
# TODO: add your solution

###################

## Part 5 (Bonus +10%): Open Challenge: Mining Biomedical Knowledge

A fundamental task in clinical/biomedical natural language processing is to extract intelligence from biomedical text corpus automatically and efficiently. More specifically, the intelligence may include biomedical entities mentioned in text, relations between biomedical entities, clinical features of patients, progression of diseases, all of which can be used to predict, understand and improve patients' outcomes. 

This open challenge is to build a biomedical knowledge graph based on the CORD-19 dataset and mine useful information from it. We recommend the following steps but you are also encouraged to develop your solution from scratch.

### Extract Biomedical Entities from Text

Extract biomedical entities (such as fever, cough, headache, lung cancer, heart attack) from text. Note that:

- The biomedical entities may consist of multiple words. For example, heart attack, multiple myeloma etc.
- The biomedical entities may be written in synoynms. For example, low blood pressure for hypotension.
- The biomedical entities may be written in different forms. For example, smoking, smokes, smoked.

### Extract Relations between Biomedical Entities

Extract relations between biomedical entities based on their appearance in text. You may define a relation between biomedical entities by one or more of the following criteria:

- The biomedical entities frequentely co-occuer together.
- The biomedical entities have similar word representations.
- The biomedical entities have clear relations based on textual narratives. For example, "The most common symptoms for COVID-19 are fever and cough" so we know there are relations between "COVID-19", "fever" and "cough".

### Build a Biomedical Knowledge Graph of COVID-19

Build a knoweledge graph based on the results from track 5.1 and 5.2 and visualise it.

In [5]:
###################
# TODO: add your solution

###################